<a href="https://colab.research.google.com/github/trainocate-japan/extending_genai_with_langchain/blob/main/openai_api_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 0: ハンズオンの準備
---

## API キーの設定
*  左ナビゲーションで [**シークレット**] アイコン (鍵形のアイコン) をクリックします。
*  [**新しいシークレットを追加**] をクリックし、`OPENAI_API_KEY` の [**値**] に指定されたキーを入力します。
*  入力が完了したら、下のセルを実行します。

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Section 1: トークン
---

## Tokenizerとtiktoken
- LLM などの自然言語処理を行う機械学習モデルは、テキストをトークンという単位で処理する。
- テキストデータなどをトークン化 (tokenize) するプログラムを **Tokenizer** という。
- トークン化の仕方や入力可能なトークン数は LLM によって異なり、入力するトークン数に応じて API 利用料金が課金される。
- **tiktoken** は OSS の高速な Tokenizer である。

In [ ]:
# !pip -q install tiktoken==0.7.0

In [ ]:
!pip -q install tiktoken

In [ ]:
!pip freeze | grep tiktoken

In [ ]:
import tiktoken

text = "It’s easy to make something cool with LLMs, but very hard to make something production-ready with them."

encoding = tiktoken.encoding_for_model("gpt-4o-mini")
tokens = encoding.encode(text)
print(len(tokens))

In [ ]:
text = "LLMを使ってクールなものを作るのは簡単だが、プロダクションで使えるものを作るのは非常に難しい。"

encoding = tiktoken.encoding_for_model("gpt-4o-mini")
tokens = encoding.encode(text)
print(len(tokens))

# Section 2: OpenAI API の利用
---

## Chat Completions API

In [ ]:
# !pip install openai==1.40.8

In [ ]:
!pip -q install openai

In [ ]:
!pip freeze | grep openai

### Chat Completions APIの呼び出し

In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello! I'm John."}
  ]
)

print(completion.choices[0].message)

### 会話履歴を踏まえた応答を得る

In [ ]:
completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! I'm John."},
        {"role": "assistant", "content": "Hello John! How can I assist you today?"},
        {"role": "user", "content": "Do you know my name?"}
    ]
)

print(completion.choices[0].message)

### ストリーミングで応答を得る

In [ ]:
stream = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
        {"role": "system", "content": "あなたは AI や機械学習に精通したプロフェッショナルです。"},
        {"role": "user", "content": "AI とは何ですか。"}
  ],
  stream=True
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

# Section 3: Function calling
---
Function Calling に対応しているモデルでは、利用可能な関数の情報をモデルに渡すことで、モデルに必要に応じた関数の利用を選択させることができる。

(※ 以下では [OpenAI の公式ドキュメント](https://platform.openai.com/docs/guides/function-calling) をもとに一部改変したコードを使用している)

## 関数を用意する  
天気予報の情報を出力する関数  
(実際に外部の天気予報サービスを利用するのではなく、既定の予報を返す疑似的な天気予報の関数)

In [ ]:
import json

def get_current_weather(location, unit="celsius"):
    weather_info = {
        "location": location,
        "temperature": "25",
        "unit": "celsius",
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

## 関数のリストを定義する
- 利用可能な関数のリストを定義する。  
- それぞれの関数については、辞書で関数名や関数についての説明、関数を呼び出す際のパラメータ等を定義する。  
- 下の例では、`tools` という名前のリストに要素として関数が 1 つだけ入っている。

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. Tokyo",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

## 関数のリストを渡して LLM を呼び出す

In [ ]:
messages = [{"role": "user", "content": "What's the weather like in Tokyo?"}]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)

print(response.choices[0].message)

In [ ]:
print(response.choices[0].message.tool_calls)

In [ ]:
print(response.choices[0].message.tool_calls[0].function.name)
print(response.choices[0].message.tool_calls[0].function.arguments)

- LLM が関数を使用することを選択した場合、使用する関数の名前や関数を呼び出す際のパラメータを回答する。
- LLM は `tools` で定義されている関数の `name` や `description` 、`parameters` を参照して回答を生成する。

## 関数を実行する  
- 上記の LLM の回答から関数名やパラメータを取得し、関数を実行する

In [ ]:
response_message = response.choices[0].message
tool_call = response_message.tool_calls[0]

available_functions = {
    "get_current_weather": get_current_weather,
}
messages.append(response_message)
function_name = tool_call.function.name
function_to_call = available_functions[function_name]
function_args = json.loads(tool_call.function.arguments)
function_response = function_to_call(
    location=function_args.get("location"),
    unit=function_args.get("unit"),
)

print(function_response)

## 関数の実行結果を `messages` に追加する

In [ ]:
messages.append({
    "tool_call_id": tool_call.id,
    "role": "tool",
    "name": function_name,
    "content": function_response,
})

## `messages` を渡して LLM を再度呼び出す

In [ ]:
second_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

print(second_response.choices[0].message)